In [2]:
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

''' 
This program merges all of the separate programs (protagonists.ipynb and synopsis.ipynb) and adds the text search function. It will
return all of the movies that contain family keywords inside a data frame. This program runs in approximately 7-8 mins."
'''

def movie_table(url):
    webpage = url
    response = requests.get(webpage)

    page = response.text

    soup = BeautifulSoup(page, 'html.parser')
    beautiful_page = soup.get_text()

    ratings = []
    years = []
    titles = []
    movie_nums = []

    lines = beautiful_page.splitlines()

    for line in lines:
        if "(20" in line and ")" in line or '(19' in line and ')' in line:
            years.append(line)

    rep_years = []
    nrep_years = []
    year_indices = []

    for year in years:
        
        if lines.count(year) > 1:
            if year not in rep_years:
                rep_years.append(year)
        if lines.count(year) == 1:
            nrep_years.append(year)

    for i in range(len(rep_years)):
        first_instance = lines.index(rep_years[i])
        year_indices.append(first_instance)

        for j, item in enumerate(lines[first_instance+1:], start=first_instance+1):
            if item == rep_years[i]:
                year_indices.append(j)

    for year in nrep_years:
        if year in lines:
            index = lines.index(year)
            year_indices.append(index)


    for index in sorted(year_indices):
        title = lines[index-1]
        titles.append(title)

        rating = lines[index+6]
        ratings.append(rating)

        movie_num = lines[index-3]
        movie_nums.append(movie_num) 
        

    ratings = [rating.lstrip() for rating in ratings]
    for i in range(len(ratings)):
        if ratings[i] == '':
            ratings[i] = "None"


    for i in range(len(years)):
        for j in range(len(years[i])):
            if years[i][j].isnumeric():
                number_index = years[i].find(years[i][j])
                years[i] = years[i][number_index:number_index+4]
                break

    table_data = list(zip(movie_nums, titles, years, ratings))
    
    return table_data

synopsis_list = []

# Function to get the synopsis from every movie in the webpage
def synopses(link):
    webpage = link
    response = requests.get(webpage)

    page = response.text

    soup = BeautifulSoup(page, 'html.parser')

    url_list = []
    movie_url_list = []
    movie_ref_nums = []

    # Get all the hyperlinks
    for link in soup.find_all('a'):
        url_list.append(link.get('href'))

    # Filter to get movie title hyperlinks
    for url in url_list:
        if '/title/tt' in url:
            if url not in movie_url_list:
                movie_url_list.append(url) 

    # Take the reference number from the movie url
    for movie_url in movie_url_list:
        if movie_url[16].isdigit():
            movie_ref_nums.append(movie_url[7:17])
        else:
            movie_ref_nums.append(movie_url[7:16])


    # Extract the synopsis from every movie then append to a list
    driver = webdriver.Chrome('/Users/nixoncarino/Desktop/chromedriver')

    for i in movie_ref_nums:
        url = "https://www.imdb.com/title/" + i + "/plotsummary/?ref_=tt_stry_pl#synopsis"
        try:
            driver.get(url)
            synopsis_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/div[2]/ul/li/div/div/div/div/div')
        except:
            synopsis_list.append("No Synopsis")
        else:
            synopsis = synopsis_element.text
            synopsis_list.append(synopsis)

origin_list = []

# Function to get the country of origin from every movie in the webpage
def origin(link):
    webpage = link
    response = requests.get(webpage)

    page = response.text

    soup = BeautifulSoup(page, 'html.parser')

    url_list = []
    movie_url_list = []
    movie_ref_nums = []

    # Get all the hyperlinks
    for link in soup.find_all('a'):
        url_list.append(link.get('href'))

    # Filter to get movie title hyperlinks
    for url in url_list:
        if '/title/tt' in url:
            if url not in movie_url_list:
                movie_url_list.append(url) 

    #Take the reference number from the movie url
    for movie_url in movie_url_list:
        if movie_url[16].isdigit():
            movie_ref_nums.append(movie_url[7:17])
        else:
            movie_ref_nums.append(movie_url[7:16])

    ##########################################################################

    #Extract the country of origin from every movie then append to a list
    driver = webdriver.Chrome('/Users/nixoncarino/Desktop/chromedriver')

    for i in movie_ref_nums:
        url = "https://www.imdb.com/title/" + i + "/?ref_=adv_li_tt"

        try:
            try:
                try:
                    try:
                        try:
                            try:
                                try:
                                    driver.get(url)
                                    origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                                except:
                                    origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                                    origin = origin_element.text
                                    if origin[0] == '$':
                                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                        origin = origin_element.text
                                        origin_list.append(origin)
                                    elif origin == 'Color':
                                        try:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                        except:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                            origin = origin_element.text
                                            origin_list.append(origin)
                                        else:
                                            origin = origin_element.text
                                            if origin[0] == '$':
                                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                                origin = origin_element.text
                                                origin_list.append(origin)
                                            else:                                
                                                origin_list.append(origin)
                                    elif origin[0:5] == 'Dolby':
                                        try:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                        except:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                            origin = origin_element.text
                                            origin_list.append(origin)
                                        else:
                                            origin = origin_element.text
                                            if origin[0] == '$':
                                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                                origin = origin_element.text
                                                origin_list.append(origin)
                                            else:                                
                                                origin_list.append(origin)
                                    elif origin[0:4] == 'Mono':
                                        try:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                        except:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                            origin = origin_element.text
                                            origin_list.append(origin)
                                        else:
                                            origin = origin_element.text
                                            if origin[0] == '$':
                                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                                origin = origin_element.text
                                                origin_list.append(origin)
                                            else:                                
                                                origin_list.append(origin)
                                    elif len(origin.split()) > 3:
                                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                                        origin = origin_element.text
                                        origin_list.append(origin)
                                    else:
                                        origin_list.append(origin)
                                else:
                                    origin = origin_element.text
                                    if origin[0] == '$':
                                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                                        origin = origin_element.text
                                        origin_list.append(origin)
                                    elif origin[0:5] == 'Color':
                                        try:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                                        except:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                            origin = origin_element.text
                                            origin_list.append(origin)
                                        else:
                                            origin = origin_element.text
                                            if origin[0] == '$':
                                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                                origin = origin_element.text
                                                origin_list.append(origin)
                                            else:                                
                                                origin_list.append(origin)
                                    elif origin[0:5] == 'Dolby':
                                        try:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                                        except:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                            origin = origin_element.text
                                            origin_list.append(origin)
                                        else:
                                            origin = origin_element.text
                                            if origin[0] == '$':
                                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                                origin = origin_element.text
                                                origin_list.append(origin)
                                            else:                                
                                                origin_list.append(origin)
                                    elif origin[0:4] == 'Mono':
                                        try:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                                        except:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                            origin = origin_element.text
                                            origin_list.append(origin)
                                        else:
                                            origin = origin_element.text
                                            if origin[0] == '$':
                                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                                origin = origin_element.text
                                                origin_list.append(origin)
                                            else:                                
                                                origin_list.append(origin)
                                    elif len(origin.split()) > 3:
                                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[12]/div[2]/ul/li[2]/div')
                                        origin = origin_element.text
                                        origin_list.append(origin)
                                    else:
                                        origin_list.append(origin)
                            except:
                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[9]/div[2]/ul/li[2]/div')
                                origin = origin_element.text
                                if origin[0] == '$':
                                    origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                    origin = origin_element.text
                                    origin_list.append(origin)
                                elif origin[0:5] == 'Color':
                                    try:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                    except:
                                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[7]/div[2]/ul/li[2]/div')
                                        origin = origin_element.text
                                        origin_list.append(origin)
                                    else:
                                        origin = origin_element.text
                                        if origin[0] == '$':
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[7]/div[2]/ul/li[2]/div')
                                            origin = origin_element.text
                                            origin_list.append(origin)
                                        else:                                
                                            origin_list.append(origin)
                                elif origin[0:5] == 'Dolby':
                                    try:
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                                    except:
                                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[7]/div[2]/ul/li[2]/div')
                                        origin = origin_element.text
                                        origin_list.append(origin)
                                    else:
                                        origin = origin_element.text
                                        if origin[0] == '$':
                                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[7]/div[2]/ul/li[2]/div')
                                            origin = origin_element.text
                                            origin_list.append(origin)
                                        else:                                
                                            origin_list.append(origin)
                        except:
                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li[2]/div')
                            origin = origin_element.text
                            if origin[0:5] == 'Color':
                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[7]/div[2]/ul/li[2]/div')
                                origin = origin_element.text
                                origin_list.append(origin)
                            elif origin[0:5] == 'Dolby':
                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[7]/div[2]/ul/li[2]/div')
                                origin = origin_element.text
                                origin_list.append(origin)
                            else:
                                origin_list.append(origin)
                    except:
                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[7]/div[2]/ul/li[2]/div')
                        origin = origin_element.text
                        if origin[0:5] == 'Color':
                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[6]/div[2]/ul/li[2]/div')
                            origin = origin_element.text
                            origin_list.append(origin)
                        elif origin[0:5] == 'Dolby':
                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[6]/div[2]/ul/li[2]/div')
                                origin = origin_element.text
                                origin_list.append(origin)
                        else:
                            origin_list.append(origin)
                except:
                    origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[13]/div[2]/ul/li[2]/div')
                    origin = origin_element.text
                    if origin[0] == '$':
                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[12]/div[2]/ul/li[2]/div')
                        origin = origin_element.text
                        origin_list.append(origin)
                    elif origin == 'Color':
                        try:
                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[12]/div[2]/ul/li[2]/div')
                        except:
                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                            origin = origin_element.text
                            origin_list.append(origin)
                        else:
                            origin = origin_element.text
                            if origin[0] == '$':
                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                                origin = origin_element.text
                                origin_list.append(origin)
                            else:                                
                                origin_list.append(origin)
                    elif origin[0:5] == 'Dolby':
                        try:
                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[12]/div[2]/ul/li[2]/div')
                        except:
                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                            origin = origin_element.text
                            origin_list.append(origin)
                        else:
                            origin = origin_element.text
                            if origin[0] == '$':
                                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                                origin = origin_element.text
                                origin_list.append(origin)
                            else:                                
                                origin_list.append(origin)
                    else:
                        origin_list.append(origin)
            except:
                origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[12]/div[2]/ul/li[2]/div')
                origin = origin_element.text
                if origin[0] == '$':
                    origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                    origin = origin_element.text
                    origin_list.append(origin)
                elif origin == 'Color':
                    try:
                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                    except:
                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                        origin = origin_element.text
                        origin_list.append(origin)
                    else:
                        origin = origin_element.text
                        if origin[0] == '$':
                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                            origin = origin_element.text
                            origin_list.append(origin)
                        else:                                
                            origin_list.append(origin)
                elif origin[0:5] == 'Dolby':
                    try:
                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[11]/div[2]/ul/li[2]/div')
                    except:
                        origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                        origin = origin_element.text
                        origin_list.append(origin)
                    else:
                        origin = origin_element.text
                        if origin[0] == '$':
                            origin_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[10]/div[2]/ul/li[2]/div')
                            origin = origin_element.text
                            origin_list.append(origin)
                        else:                                
                            origin_list.append(origin)
                else:
                    origin_list.append(origin)    
        except:
            origin_list.append("Unknown")

    # Format list

    countries = ['United States', 'France', 'United Kingdom', 'Canada', 'Japan', 'Denmark', 'Australia', 'Italy', 
                'Germany', 'Spain', 'India', 'China', 'Mexico', 'Lithuania', 'Sweden', 'Iceland', 'New Zealand', 
                'Malta', 'Morocco', 'Ireland', 'Hungary', 'Norway', 'Finland', 'Turkey', "Belgium", 'South Korea',
                'United Arab Emirates', 'Ireland', 'Greece', 'Switzerland', 'Croatia', 'Serbia', 'Netherlands',
                'South Africa', 'Hong Kong', 'West Germany', 'Chile', 'Israel']

    for i in range(len(origin_list)):
        for country in countries:
            instance_index = origin_list[i].find(country)
            if instance_index == -1:
                continue
            try:
                to_check = origin_list[i][instance_index + len(country)]
            except IndexError:
                continue
            else:
                if country in origin_list[i] and to_check.isalpha():
                    origin_list[i] = origin_list[i].replace(country, f"{country}, ")



# Function that find family related words on the synopsis
def text_search(synopsis_list):
    # The indices of the synopses that contain any of the keywords
    match_index_list = []

    keywords = ["family", "parenting", "father", "mother", "son", "daughter", "sibling", "brother", "sister", "marriage", "divorce", "blended family", "adoption",
                "foster care", "custody", "inheritance", "legacy", "generations", "multigenerational", "family reunion", "family secrets", "family drama", "dysfunctional", 
                "dysfunctional family", "family conflict", "family relationships", "family values", "family ties", "parent-child relationship", "relationship",
                "coming-of-age", "parenthood", "growing up", "childhood", "teenage years", "aging parents", "grandparents"]

    for i in range(len(synopsis_list)):
        for keyword in keywords:
            if keyword in synopsis_list[i]:
                if i not in match_index_list:
                    match_index_list.append(i)

    return match_index_list

def non_family_movies(synopsis_list):
    all_indices = list(range(0,len(synopsis_list)))

    # The indices of the synopses that contain any of the keywords
    match_index_list = []

    keywords = ["family", "parenting", "father", "mother", "son", "daughter", "sibling", "brother", "sister", "marriage", "divorce", "blended family", "adoption",
                "foster care", "custody", "inheritance", "legacy", "generations", "multigenerational", "family reunion", "family secrets", "family drama", "dysfunctional", 
                "dysfunctional family", "family conflict", "family relationships", "family values", "family ties", "parent-child relationship", "relationship",
                "coming-of-age", "parenthood", "growing up", "childhood", "teenage years", "aging parents", "grandparents"]

    for i in range(len(synopsis_list)):
        for keyword in keywords:
            if keyword in synopsis_list[i]:
                if i not in match_index_list:
                    match_index_list.append(i)
    
    non_match_index = [index for index in all_indices if index not in match_index_list]

    return non_match_index

# Call synopses function to put all synopses into list
synopses("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=adventure&view=simple")
synopses("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=adventure&view=simple&start=51&ref_=adv_nxt")

# Call origin function to put all countries of origin into list
origin("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=adventure&view=simple")
origin("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=adventure&view=simple&start=51&ref_=adv_nxt")

# Create Data Frame for Movie information
df_animation_1 = pd.DataFrame(movie_table("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=adventure&view=simple"), 
                            columns=['Index', 'Title', 'Year', 'Rating'])
df_animation_2 = pd.DataFrame(movie_table("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=adventure&view=simple&start=51&ref_=adv_nxt"), 
                            columns=['Index', 'Title', 'Year', 'Rating'])
df_animation = pd.concat([df_animation_1, df_animation_2], ignore_index=True).assign(Genre='Adventure', Synopsis=synopsis_list, Origin=origin_list)

# Output information of movies that contain family keywords
pd.set_option('display.max_rows', None)

#If you want to see the movies that contain the keyword: change the argument inside iloc to text_search(synopsis_list)


/var/folders/dr/q8lk9535757d9krftyrx8kgc0000gn/T/ipykernel_64706/3322560295.py:122: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/nixoncarino/Desktop/chromedriver')


In [3]:
family = df_animation.iloc[text_search(synopsis_list)]; family

,Index,Title,Year,Rating,Genre,Synopsis
0,1.,Guardians of the Galaxy Vol. 3,2023,8.3,Adventure,"As a baby raccoon, Rocket is a test subject fo..."
1,2.,Dungeons & Dragons: Honor Among Thieves,2023,7.4,Adventure,"Prior to imprisonment, bard Edgin Darvis serve..."
3,4.,Guardians of the Galaxy,2014,8,Adventure,"Earth, 1988:\n\nYoung Peter Quill (Wyatt Oleff..."
5,6.,Guardians of the Galaxy Vol. 2,2017,7.6,Adventure,"Missouri, Planet Earth, 1980\n\nMeredith Quill..."
7,8.,Dune,2021,8,Adventure,The story opens with a woman telling a portion...
8,9.,Avatar: The Way of Water,2022,7.7,Adventure,Over a decade after the events of the first fi...
9,10.,Everything Everywhere All at Once,2022,7.9,Adventure,Part 1: Everything\n\nEvelyn Wang (Yeoh) is a ...
11,12.,The Suicide Squad,2021,7.2,Adventure,The film opens at Belle Reve Penitentiary wher...
12,13.,Interstellar,2014,8.6,Adventure,A group of elderly people are giving interview...
15,16.,Avengers: Endgame,2019,8.4,Adventure,"In the opening, Clint Barton is teaching his d..."


In [4]:
non_family = df_animation.iloc[non_family_movies(synopsis_list)]; non_family

,Index,Title,Year,Rating,Genre,Synopsis
2,3.,The Super Mario Bros. Movie,2023,7.3,Adventure,No Synopsis
4,5.,Ghosted,2023,5.8,Adventure,No Synopsis
6,7.,Ant-Man and the Wasp: Quantumania,2023,6.2,Adventure,No Synopsis
10,11.,Shazam! Fury of the Gods,2023,6,Adventure,No Synopsis
13,14.,65,2023,5.4,Adventure,No Synopsis
14,15.,Puss in Boots: The Last Wish,2022,7.9,Adventure,No Synopsis
16,17.,Thor: Love and Thunder,2022,6.2,Adventure,No Synopsis
17,18.,Spider-Man: No Way Home,2021,8.2,Adventure,No Synopsis
18,19.,Peter Pan,2003,6.8,Adventure,No Synopsis
32,33.,The Goonies,1985,7.7,Adventure,No Synopsis
